In [1]:
import datajoint as dj
import numpy as np
import time

from meshparty import trimesh_io

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.FilteredSkeleton()
#ta3p100.NeuriteRawSkeletonSpan()

In [4]:
# #how to extract the bounding box

# key = dict(segmentation=2,segment_id=648518346341351731)
# skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
# print(skeleton_edges.shape)
# vertices = skeleton_edges.reshape(-1,3)
# vertices
# mins = np.amin(vertices, axis=0)
# maxs = np.amax(vertices, axis=0)
# print("mins,maxs = " + str((mins,maxs)))
# diff = maxs-mins
# volume = diff[0]*diff[1]*diff[2]
# print("volume = " + str(volume/1000000000))

In [5]:
# #how to extract the lengths
# filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  skeleton_edges]
# sum(filtered_edges_distance_lin_alg)

In [6]:
#ta3p100.Synapse.describe()

In [7]:
@schema
class NeuriteRawSkeletonSpan(dj.Computed):
    definition="""
    -> ta3p100.FilteredSkeleton
    ---
    bbox_volume   :double unsigned #length of the entire skeleton, raw_volume/10^9
    skeleton_length    :double unsigned #length of the entire skeleton
    skel_bbox_x1          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_y1          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_z1          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_x2          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_y2          : bigint unsigned              # (EM voxels)  - bounding box
    skel_bbox_z2          : bigint unsigned              # (EM voxels)  - bounding box
    size                 : bigint unsigned              # (EM voxels)
    """
    
    def make(self, key):
        start_time = time.time()
        print(str(key["segment_id"]) + ":")
        #get the skeleton edges
        skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
        vertices = skeleton_edges.reshape(-1,3)
        
        mins = np.amin(vertices, axis=0)
        maxs = np.amax(vertices, axis=0)
        #print("mins,maxs = " + str((mins,maxs)))
        #key["bbox_corners"] = (mins,maxs)
        key["skel_bbox_x1"] = mins[0]
        key["skel_bbox_y1"] = mins[1]
        key["skel_bbox_z1"] = mins[2]
        key["skel_bbox_x2"] = maxs[0]
        key["skel_bbox_y2"] = maxs[1]
        key["skel_bbox_z2"] = maxs[2]

        
        diff = maxs-mins
        volume = diff[0]*diff[1]*diff[2]
        #print("volume = " + str(volume/1000000000))
        key["bbox_volume"] = volume/1000000000
        
        filtered_edges_distance_lin_alg = [np.linalg.norm(a[0]-a[1]) for a in  skeleton_edges]
        total_length = sum(filtered_edges_distance_lin_alg)
        
        key["skeleton_length"] = total_length
        
        self.insert1(key,skip_duplicates=True)
        
        print(f"Total Time: {time.time()-start_time}")
        print()
        
    
    

In [8]:
#ta3p100.NeuriteRawSkeletonReach.drop()

In [9]:
#(schema.jobs & "table_name='__neurite_raw_skeleton_reach'").delete()

In [10]:
start_time = time.time()
NeuriteRawSkeletonSpan.populate(reserve_jobs=True)
print(f"Total time for populate: {time.time()-start_time}")

648518346341362194:
Total Time: 0.0410914421081543

648518346341362268:
Total Time: 0.01469731330871582

648518346341362271:
Total Time: 0.031313180923461914

648518346341362283:
Total Time: 0.037404537200927734

648518346341362287:
Total Time: 0.029446125030517578

648518346341362301:
Total Time: 0.03529500961303711

648518346341362302:
Total Time: 0.018403053283691406

648518346341362315:
Total Time: 0.043427228927612305

648518346341362316:
Total Time: 0.05075883865356445

648518346341362364:
Total Time: 0.03539466857910156

648518346341362451:
Total Time: 0.040563106536865234

648518346341362454:
Total Time: 0.024410009384155273

648518346341362455:
Total Time: 0.03330588340759277

648518346341362460:
Total Time: 0.026417970657348633

648518346341362461:
Total Time: 0.030308008193969727

648518346341362473:
Total Time: 0.03217935562133789

648518346341362474:
Total Time: 0.04183840751647949

648518346341362505:
Total Time: 0.02902078628540039

648518346341362515:
Total Time: 0.0436

OperationalError: (1213, 'Deadlock found when trying to get lock; try restarting transaction')